In [17]:
import fastplotlib as fpl
import os
import sys
import masknmf
import tifffile
import numpy as np
%load_ext autoreload

In [2]:
data = tifffile.imread("../../demo_data/demoMovie.tif")

# If you don't have a good template estimate, run the generic template estimation procedure. End result is a PiecewiseRigidRegistrationStrategy object, used to register all frames to a template

In [18]:
pwrigid_strategy = masknmf.PiecewiseRigidMotionCorrector(
    num_blocks=[10, 10],
    overlaps=[5, 5],
    max_rigid_shifts=[15, 15],
    max_deviation_rigid=[2, 2],
    batch_size=500
)

pwrigid_strategy.compute_template(data)

# Define a RegistrationArray that lazily loads motion corrected frames of the raw data

In [4]:
moco_results = masknmf.RegistrationArray(data, pwrigid_strategy)

# Visualize the raw data, motion corrected data, template

In [12]:
iw = fpl.ImageWidget(data = [data, moco_results],
                     names = ['raw', 'motion corrected'])
iw.show()

## Access the shifts per block per frame

In [14]:
shifts = moco_results.shifts[:]

# Visualize with fastplotlib imagewidget (will be updated with next fastplotlib update)

In [16]:
# iw = fpl.ImageWidget(
#     data=[data, moco_results, pwrigid_strategy.template],
#     names = ['raw data', 'motion corrected', 'template'],
#     figure_shape=(1, 3),
#     cmap="viridis",
#     window_funcs={"t": (np.mean, 11)},
# )

# x, y = moco_results.block_centers.transpose(-1, 0, 1)
# u, v = moco_results.shifts[0].transpose(-1, 0, 1)

# # positions of each vector as [n_points, 2] array
# positions = np.column_stack([x.ravel(), y.ravel()])

# # directions of each vector as a [n_points, 2] array
# # scale down by 5 otherwise they're too big
# directions = np.column_stack([u.ravel(), v.ravel()]) / 5

# vector_field = iw.figure[0, 0].add_vector_field(
#     positions=positions,
#     directions=directions,
#     alpha=0.7,
#     alpha_mode="add",
#     color="w",
# )

# @iw.add_event_handler
# def update_vector_field(index):
#     t = index["t"]

#     u, v = moco_results.shifts[t].transpose(-1, 0, 1)
#     directions = np.column_stack([u.ravel(), v.ravel()])
    
#     vector_field.directions = directions / 5


# iw.show()